<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221020_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

com.adacho.mapper 패키지에

ArrivalDelayCountMapper 클래스 생성

In [ ]:
package com.adacho.mapper;

import java.io.IOException;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

import com.adacho.common.AirlinePerformanceParser;

public class ArrivalDelayCountMapper extends Mapper<LongWritable, Text, Text, IntWritable>{
	   private final static IntWritable outputValue = new IntWritable(1);
	   private Text outputKey = new Text();
	   
	   @Override
	   protected void map(LongWritable key, Text value, Mapper<LongWritable, Text, Text, IntWritable>.Context context)
	         throws IOException, InterruptedException {
	      // TODO Auto-generated method stub
	      AirlinePerformanceParser parser = new AirlinePerformanceParser(value);
	      
	      outputKey.set(parser.getYear() + "," + parser.getMonth());
	      
	      if(parser.getArriveDelayTime() >0 ) {
	         context.write(outputKey, outputValue);
	      }
	   }
	}


com.adacho.driver 패키지에

ArrivalDelayCount 클래스 생성


In [ ]:
package com.adacho.driver;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.mapreduce.lib.output.TextOutputFormat;

import com.adacho.mapper.ArrivalDelayCountMapper;
import com.adacho.reducer.DelayCountReducer;

public class ArrivalDelayCount {
	public static void main(String[] args) throws Exception{

		Configuration conf = new Configuration();
		if(args.length !=2) {
			System.err.println("Usage: ArrivalDelayCount <input> <output>");
			System.exit(1);
		}

		Job job = Job.getInstance(conf, "ArrivalDelayCount");

		job.setJarByClass(ArrivalDelayCount.class); //드라이브 클래스 저장
		job.setMapperClass(ArrivalDelayCountMapper.class); //메퍼클래스저장
		job.setReducerClass(DelayCountReducer.class); //리듀스클래스저장

		job.setInputFormatClass(TextInputFormat.class); //두개중에 길이가 긴 것 임포트
		job.setOutputFormatClass(TextOutputFormat.class); //두개중에 길이가 긴 것 임포트

		job.setMapOutputKeyClass(Text.class);
		job.setMapOutputValueClass(IntWritable.class);

		FileInputFormat.addInputPath(job, new Path(args[0])); //여러개에 인풋데이타가 있으면 다 줄 수 있음
		FileOutputFormat.setOutputPath(job, new Path(args[1]));

		job.waitForCompletion(true); //job이끝날때까지 기다리고있다가 동기방식으로 

	}
}


메이븐 클린

메이븐 인스톨 (jar 오버라이트됨)

자르 경로복사해서

cd 자르경로

scp .\airPerformance-0.1.jar ubuntu:~/work/java  윈도우에서 리눅스로 

/work/java에서

yarn jar ./airPerformance-0.1.jar com.adacho.driver.ArrivalDelayCount air-input arr-delay-count

hdfs dfs -ls arr-delay-count

hdfs dfs -cat arr-delay-count/part-r-00000